<a href="https://colab.research.google.com/github/studorache/MLC/blob/master/MLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

#download data
!kaggle competitions download -c ml-challenge-week6

#unzip data
!unzip census-income.data.zip
!unzip census-income.test.zip

#create panda dataframe from training data
import pandas as pd 



In [0]:
initial_train_df = pd.read_csv('census-income.data', header=None,sep=',')
final_test_df = pd.read_csv('census-income.test', header=None,sep=',')

In [0]:
#get cols names
lines = open("census-income.names", "r").readlines()
i = 0
col_names = []
while i < len(lines):
    if i >= 81 and i<= 120:
      s = lines[i]
      col_names.append(s[s.find("(")+1:s.find(")")])
    i += 1
col_names.insert(col_names.index('detailed household summary in household')+1, 'instance weight') 

In [0]:
#Append cols list to test panda
final_test_df.columns = col_names

col_names.append('income class')
#Append cols list to train panda
initial_train_df.columns = col_names

In [0]:
categories_to_encode = ["class of worker", "education", "enroll in edu inst last wk", "marital stat", 
                       "major industry code", "major occupation code", "race", "hispanic origin", "sex", 
                       "member of a labor union", "reason for unemployment", "full or part time employment stat", 
                       "tax filer stat", "region of previous residence", "state of previous residence", 
                       "detailed household and family stat", "detailed household summary in household", 
                       "migration code-change in msa", "migration code-change in reg", 
                       "migration code-move within reg", "live in this house 1 year ago", 
                       "migration prev res in sunbelt", "family members under 18", "country of birth father", 
                       "country of birth mother", "country of birth self", "citizenship", 
                       "fill inc questionnaire for veteran's admin" ]

for cat in categories_to_encode:
  initial_train_df[cat] = initial_train_df[cat].astype('category')
  initial_train_df[cat] = initial_train_df[cat].cat.codes

for cat in categories_to_encode:
  final_test_df[cat] = final_test_df[cat].astype('category')
  final_test_df[cat] = final_test_df[cat].cat.codes


initial_train_df["income class"] = initial_train_df["income class"].astype('category')
initial_train_df["income class"] = initial_train_df["income class"].cat.codes


In [0]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(initial_train_df, test_size=0.2, random_state=42)

In [0]:

X_train = train_set.drop(['income class'], axis=1)
y_train =train_set['income class']


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Create the parameter grid
param_grid = {'max_depth': list(range(5,26)), 'max_features': ["auto", "sqrt", "log2", None]} 

# Create a random search object
model = RandomizedSearchCV(
    estimator = RandomForestClassifier(n_estimators=80, random_state=42),
    param_distributions = param_grid, n_iter = 5,
    scoring='roc_auc', n_jobs=4, cv = 3, refit=True, return_train_score = True)



In [0]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=5, min_samples_leaf=5)
# Fit to the training data
model.fit(X_train, y_train)

In [0]:


from sklearn.metrics import accuracy_score, confusion_matrix



In [0]:
X_test = test_df
X_pred = model.predict(X_test)

In [0]:
result = pd.DataFrame(X_pred)
#result.insert(0, 'index', result.index)
result.columns= ['income class']
result.to_csv('result.csv', index=True, index_label='index')

In [0]:
!kaggle competitions submit -c ml-challenge-week6 -f result.csv -m "First try."

100% 769k/769k [00:01<00:00, 645kB/s] 
Successfully submitted to ML Challenge